<a href="https://colab.research.google.com/github/sarvesh-kale/ML-assignment-/blob/master/Copy_of_Copy_of_ML_assgn_Dataloading_sar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
from PIL import Image  
import matplotlib.pyplot as plt
!pip install pydicom
import pydicom
import csv
import numpy as np
import cv2
from google.colab.patches import cv2_imshow
import torch
from torch.utils.data import DataLoader,random_split
import torchvision.transforms.functional as tr_F

     |████████████████████████████████| 35.5MB 84kB/s 


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#%cd drive
%cd My \Drive

**TRANFORMS**

In [ ]:
class ToTensorNormalize(object):
    """
    Convert images/masks to torch.Tensor
    Scale images' pixel values to [0-1] and normalize with predefined statistics
    """
    def __call__(self, sample):
        img = sample['image']
        img = tr_F.to_tensor(img)
        img = tr_F.normalize(img, mean=0.5, std=0.5)
        sample['image'] = img
        return sample
   

**PREPARING INPUT**

In [ ]:
def get_file_paths(root,line): # line = line read from csv file
        file_name='-'.join(line.split('\\')[-1].split('-')[0:2])
        i_contour_name=file_name+'-icontour-manual.txt'
        o_contour_name=file_name+'-ocontour-manual.txt'
        image_name=file_name+'.dcm'

        contour_root=root+'/'+'/'.join(line.split('\\')[1:3])
        image_root=root+'/'+(line.split('\\')[1])+'/'+(file_name[0:3])+'dicom'

        i_path=contour_root+'/'+i_contour_name
        o_path=contour_root+'/'+o_contour_name
        image_path=image_root+'/'+image_name
        #print(image_path)
        
        return image_path,i_path,o_path

class RVDataset():
    """
    Base Class for RV Dataset

    Args:
        base_dir:
            RV dataset directory
        split:
            which split to use
            choose from ('train', 'val', 'trainval', 'trainaug')
        transform:
            transformations to be performed on images/masks
        to_tensor:
            transformation to convert PIL Image to tensor
    """
    def __init__(self, base_dir, transforms=None, to_tensor=None):
        
        self.base_dir= base_dir
        self.to_tensor = to_tensor
        f=open(self.base_dir + "/P_list.txt",mode="r")                                     ## Check the code syntax with example
        self.lines=[]
        for i, dummy in enumerate(f.readlines()):
            if i % 2 == 0:
                self.lines.append(dummy)

    def __len__(self):
        return len(self.lines)
 
    
    def __getitem__(self, idx):
        # Fetch data

        line=self.lines[idx]
        
        image_path, epicardium_path, endocardium_path = get_file_paths(self.base_dir,line)
        image_detail=pydicom.dcmread(image_path)                                            
  
        x=[]
        y=[]
        with open(epicardium_path,'r') as f:
          reader = csv.reader(f,delimiter=' ')
          reader = np.array(list(reader), dtype=float)
        for count,line in enumerate(reader):
          x.append(line[0])
          y.append(line[1])
        ic = np.transpose(np.vstack((x,y))) 

        x=[]
        y=[]
        with open(endocardium_path,'r') as f:
          reader = csv.reader(f,delimiter=' ')
          reader = np.array(list(reader), dtype=float)
        for line in reader:
          x.append(line[0])
          y.append(line[1])        
        oc = np.transpose(np.vstack((x,y)))
        
        image = image_detail.pixel_array
        image=image.astype('uint8')
        

        image_dummy=image.copy()
        ctr_i = np.array(ic).reshape((-1,1,2)).astype(np.int32)
        ctr_o = np.array(oc).reshape((-1,1,2)).astype(np.int32)              

        sizz = image.shape
        image_dummy2 = np.zeros(sizz, dtype=float) 
        mask_1 =   cv2.fillPoly(image_dummy2,[ctr_i],[255,255,255])
        mask_1 = mask_1/255

        image_dummy2 = np.zeros(sizz, dtype=float ) 
        mask_2 =   cv2.fillPoly(image_dummy2,[ctr_o],[255,255,255]) 
        mask_2 = mask_2/255
        
        if(image.shape == (256,216)):
          image = np.transpose(image)
          mask_1 = np.transpose(mask_1)
          mask_2 = np.transpose(mask_2)


        mask_1 = torch.Tensor(np.array(mask_1))   #long()   
        mask_2 = torch.Tensor(np.array(mask_2))   #long()
        target = torch.stack ((mask_1, mask_2),dim=0)


        sample = {'image': image,
                  'target': target
                  }

        # Transform to tensor
        if self.to_tensor is not None:
            sample = self.to_tensor(sample)


        return sample['image'],sample['target']

   


**TRYING OUT INPUT**

In [ ]:
import math
%cd /content/drive/My \Drive
dataset = RVDataset(
        base_dir="/content/drive/My Drive/Training",to_tensor=ToTensorNormalize(),)
#creating validation and train datasets          
batch_size = 16
print(len(dataset))
train_data,valid_data = random_split(dataset,[math.ceil(len(dataset)*0.9),math.floor(len(dataset)*0.1)])
train_loader = DataLoader(train_data,batch_size=batch_size,shuffle=True,drop_last=True,num_workers=8)
val_loader = DataLoader(valid_data,batch_size=1,shuffle=True,drop_last=True,num_workers=8)


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class DoubleConv(nn.Module):
    """(convolution => [BN] => ReLU) * 2"""

    def __init__(self, in_channels, out_channels, mid_channels=None):
        super().__init__()
        if not mid_channels:
            mid_channels = out_channels
        self.double_conv = nn.Sequential(
            nn.Conv2d(in_channels, mid_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(mid_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(mid_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        return self.double_conv(x)


class Down(nn.Module):
    """Downscaling with maxpool then double conv"""

    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.maxpool_conv = nn.Sequential(
            nn.MaxPool2d(2),
            DoubleConv(in_channels, out_channels)
        )

    def forward(self, x):
        return self.maxpool_conv(x)


class Up(nn.Module):
    """Upscaling then double conv"""

    def __init__(self, in_channels, out_channels, bilinear=True):
        super().__init__()

        # if bilinear, use the normal convolutions to reduce the number of channels
        if bilinear:
            self.up = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
            self.conv = DoubleConv(in_channels, out_channels, in_channels // 2)
        else:
            self.up = nn.ConvTranspose2d(in_channels , in_channels // 2, kernel_size=2, stride=2)
            self.conv = DoubleConv(in_channels, out_channels)


    def forward(self, x1, x2):
        x1 = self.up(x1)
        # input is CHW
        diffY = x2.size()[2] - x1.size()[2]
        diffX = x2.size()[3] - x1.size()[3]

        x1 = F.pad(x1, [diffX // 2, diffX - diffX // 2,
                        diffY // 2, diffY - diffY // 2])
       
        x = torch.cat([x2, x1], dim=1)
        return self.conv(x)


class OutConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(OutConv, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size=1)

    def forward(self, x):
        return self.conv(x)

In [ ]:
import torch.nn.functional as F



class UNet(nn.Module):
    def __init__(self, n_channels, n_classes, bilinear=True):
        super(UNet, self).__init__()
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.bilinear = bilinear

        self.inc = DoubleConv(n_channels, 64)
        self.down1 = Down(64, 128)
        self.down2 = Down(128, 256)
        self.down3 = Down(256, 512)
        factor = 2 if bilinear else 1
        self.down4 = Down(512, 1024 // factor)
        self.up1 = Up(1024, 512 // factor, bilinear)
        self.up2 = Up(512, 256 // factor, bilinear)
        self.up3 = Up(256, 128 // factor, bilinear)
        self.up4 = Up(128, 64, bilinear)
        self.outc = OutConv(64, n_classes)

    def forward(self, x):
        x1 = self.inc(x)
        x2 = self.down1(x1)
        x3 = self.down2(x2)
        x4 = self.down3(x3)
        x5 = self.down4(x4)
        x = self.up1(x5, x4)
        x = self.up2(x, x3)
        x = self.up3(x, x2)
        x = self.up4(x, x1)
        logits = self.outc(x)
        return logits
    
    def init_weight(self):
        for m in self.modules():
          if isinstance(m,nn.Conv2d):
            torch.nn.init =xavier_uniform(m.weight,gain=nn.init.calculate_gain('relu'))
      

 **MODEL**

In [ ]:
loss_bce = nn.BCELoss()


#dice loss
class DiceLoss(nn.Module):
    def __init__(self, weight=None, size_average=True):
        super(DiceLoss, self).__init__()

    def forward(self, inputs, targets, smooth=1):
        
        #comment out if your model contains a sigmoid or equivalent activation layer
        inputs = F.sigmoid(inputs)       
        
        #flatten label and prediction tensors
        inputs = inputs.view(-1)
        targets = targets.view(-1)
        
        intersection = (inputs * targets).sum()                            
        dice = (2.*intersection + smooth)/(inputs.sum() + targets.sum() + smooth)  
        
        return 1 - dice

class DiceBCELoss(nn.Module):
    def __init__(self, weight=None, size_average=True):
        super(DiceBCELoss, self).__init__()

    def forward(self, inputs, targets, smooth=1):
        
        #comment out if your model contains a sigmoid or equivalent activation layer
        inputs = F.sigmoid(inputs)       
        
        #flatten label and prediction tensors
        inputs = inputs.view(-1)
        targets = targets.view(-1)
        
        intersection = (inputs * targets).sum()                            
        dice_loss = 1 - (2.*intersection + smooth)/(inputs.sum() + targets.sum() + smooth)  
        BCE = F.binary_cross_entropy(inputs, targets, reduction='mean')
        Dice_BCE = BCE + dice_loss
        
        return Dice_BCE


#PyTorch
ALPHA = 0.8
GAMMA = 2

class FocalLoss(nn.Module):
    def __init__(self, weight=None, size_average=True):
        super(FocalLoss, self).__init__()

    def forward(self, inputs, targets, alpha=ALPHA, gamma=GAMMA, smooth=1):
        
        #comment out if your model contains a sigmoid or equivalent activation layer
        inputs = F.sigmoid(inputs)       
        
        #flatten label and prediction tensors
        inputs = inputs.view(-1)
        targets = targets.view(-1)
        
        #first compute binary cross-entropy 
        BCE = F.binary_cross_entropy(inputs, targets, reduction='mean')
        BCE_EXP = torch.exp(-BCE)
        focal_loss = alpha * (1-BCE_EXP)**gamma * BCE
                       
        return focal_loss



**GPU Procesing**

In [ ]:
torch.cuda.is_available()
x='cuda' if torch.cuda.is_available() else 'cpu'

**OPTIMIZER**

In [ ]:
model = UNet(1,1) #this line corrected an error 
model=model.to(x)

In [ ]:
from torch import optim
o_adam = optim.Adam(model.parameters(),lr=0.0001,weight_decay=0.01)
scheduler=optim.lr_scheduler.CosineAnnealingLR(o_adam, T_max=25, eta_min=0.0001)

**INSTANCE OF  MODEL**

**TRAINING SCRIPT**

**ADDITIONAL TRAINING SCRIPT**

In [ ]:
%pwd

model.train()
%cd /content/drive/My \Drive
i_iter = 0

                           
for input,target in train_loader:

    # Prepare input

    input = input.view(batch_size,-1,216,256)
    input=input.to(x)
    tar1,tar2 = torch.split(target,[1,1],1)
    
    tar1=tar1.view(batch_size,-1,216,256)
    tar2=tar2.view(batch_size,-1,216,256)
    tar1=tar1.to(x)
    tar2=tar2.to(x)
    i_iter+=1
    print(input.shape,tar1.shape)

    # Forward and Backward

    o_adam.zero_grad()
    pred1 = model(input.cuda())
    pred2 = model(input.cuda())
    loss1 = loss_bce(pred1.cuda(),tar1.cuda())
    loss2 = loss_bce(pred2.cuda(),tar2.cuda())
    print(loss1,loss2)
    loss=(loss1+loss2)/2
    loss.backward()
    o_adam.step()
    scheduler.step()

    if (i_iter + 1)% 5 == 0:
        
        torch.save(model.state_dict(),
                  os.path.join('https://drive.google.com/drive/folders/0B3rtY00URrIuYnhfOFZKY3JZRkE?usp=sharing', 'save_weights.pth'))

torch.save(model.state_dict(),
          os.path.join('https://drive.google.com/drive/folders/0B3rtY00URrIuYnhfOFZKY3JZRkE?usp=sharing', 'save_weights.pth'))


In [ ]:
model.eval()
loss=0 
for input,target in valid_loader:

    input = target.view(batch_size,-1,216,256)
    input=input.to(x)
    target = target.view(batch_size,216,256)
    target=target.to(x)
    prediction = model(input)

    loss=loss_bce(prediction,target)
    print(loss)

    

In [ ]:
# maskA,maskB = torch.split(prediction,[1,1],1)
ta,tb = torch.split(target,[1,1],1)

maskA1=(maskA>0.5).float()
plt.imshow(ta.squeeze().detach().cpu().numpy(),cmap='gray')

plt.figure()
plt.imshow(maskA1.squeeze().detach().cpu().numpy(),cmap='gray')
